In [1]:
##Robs summariser of RC9 Additional Scenario Regional Contributor tables
import pandas as pd
import numpy as np
import libarchive
import tempfile
import os
from functools import reduce

#mainPath = 'P:\projects\RC9_ResultsSets\Tester'
mainPath = r'\\athenasmb\pdreef\RC9_RC2017\AdditionalScenarios\Results\ReportCardSpan_1986_2014'
#mainPath = r'\\athenasmb\pdreef\RC9_RC2017\RC9_ResultsSets_PointOfTruth'
#outPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
outPath = 'P:\projects\RC9_ResultsSets\Tester\out'
summariesOutFolder = 'Summaries'
baseFileName = 'ScenarioBaseline'
D2CFileName = 'Scenario_DtoC'
AllBFileName = 'Scenario_AllB'
AllAFileName = 'Scenario_AllA'

regContributorFileName = 'RegContributorDataGrid.csv'
fuAreasFileName = 'fuAreasTable.csv'
#outputCSVEnd = 'RegToOutlet_Alt.csv'
outputCSVEnd = 'RegToOutlet.csv'
baselineScenarioName = 'Baseline'
changeScenarioName = 'Change'
predevScenarioName = 'PreDev'
reportCardString = 'RC2017'

preDevCSVLoc = 'P:\projects\RC9_ResultsSets\RegContribTables\Summaries'

theTempDir = tempfile.gettempdir()
theCurrentDir = os.getcwd()

print("Current dir is: " + theCurrentDir)

pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC9_RC2017\RC9_ResultsSets_PointOfTruth\constituentsToGroups.csv'

annLoadToStream = 'AnnLoadToStreamKG'
annLoadToExport = 'AnnLoadToExportKG'

scenarioList = [baseFileName, D2CFileName, AllBFileName, AllAFileName]
regList = ['BM','CY','FI','MW','WT']
#scenarioToFolderDict = {baselineScenarioName:baseFolderEnd, changeScenarioName:changeFolderEnd, predevScenarioName:predevFolderEnd}


Current dir is: C:\PythonScriptingForGBR\reef_scripts\ResultsAnalysis


In [2]:
def getPathInfo(regionIDString, scenarioString):
    #fileIn = mainPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_' + scenarioToFolderDict[scenarioString] + '\\' + regContributorFileName
    #fileOut = outPath + '\\' + summariesOutFolder + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    fileIn = mainPath + '\\' + regionIDString + '_RC9_' + scenarioString + '_1986_2014.7z'
    fileOut = outPath + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut


In [3]:
def produceSummaryFromRegContrib(regionIDString, scenarioName):
    
    theFileIn, theFileOut = getPathInfo(regionIDString, scenarioName)
    
    print("Processing " + theFileIn + " Scenario: " + scenarioName)
    
    with libarchive.file_reader(theFileIn) as e:
        for entry in e:
            if(regContributorFileName in entry.pathname):
                print(entry)
                with open(theTempDir + "\\" + regContributorFileName, 'wb') as f:
                    for block in entry.get_blocks():
                        f.write(block)
                    break
    
    rawcontrib = pd.read_csv(theTempDir + "\\" + regContributorFileName)
    
    rawcontrib['SCENARIO'] = scenarioName
    rawcontrib[annLoadToStream] = rawcontrib['LoadToStream (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib[annLoadToExport] = rawcontrib['LoadToRegExport (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib['AreaHA'] = rawcontrib['AreaM2'].div(10000)
    
    ### Can keep reporting region in the area table, as this is our Non-Nested notebook
    catchFuArea = pd.DataFrame(rawcontrib.groupby(['Rep_Region','ModelElement','FU']).agg({'AreaHA':'first'})).reset_index()
    regFuArea = pd.DataFrame(catchFuArea.groupby(['Rep_Region','FU']).agg({'AreaHA':'sum'})).reset_index()
    
    #regLuseSummary = pd.DataFrame(rawcontrib[rawcontrib['Constituent'].isin(selectedConstituents)].groupby(['Rep_Region','Constituent','FU','Process']).agg({'AnnLoadToStreamKG':'sum','AnnLoadToExportKG':'sum'})).reset_index()
    regLuseSummary = pd.DataFrame(rawcontrib.groupby(['SCENARIO','Rep_Region','Constituent','FU','Process']).agg({annLoadToStream:'sum',annLoadToExport:'sum'})).reset_index()
    
    ### Join/merge with Regional Areas
    regLuseSumPlusArea = pd.merge(regLuseSummary, regFuArea, how='left', left_on=['Rep_Region','FU'], right_on = ['Rep_Region','FU'])
    
    regLuseSumPlusArea.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [4]:
#List out the processes

for regID in regList:
    for scen in scenarioList:
        produceSummaryFromRegContrib(regID, scen)



##Burdekin
#regID = 'BU'
#produceSummaryFromRegContrib(regID, baselineScenarioName)
#produceSummaryFromRegContrib(regID, changeScenarioName)
#produceSummaryFromRegContrib(regID, predevScenarioName)


#Burnett Mary
#regID = 'BM'
##produceSummaryFromRegContrib(regID, baseFileName)
#for scen in scenarioList:
#    produceSummaryFromRegContrib(regID, scen)

#produceSummaryFromRegContrib(regID, baselineScenarioName)
#produceSummaryFromRegContrib(regID, changeScenarioName)
#produceSummaryFromRegContrib(regID, predevScenarioName)

##Cape York
#regID = 'CY'
#produceSummaryFromRegContrib(regID, baselineScenarioName)
#produceSummaryFromRegContrib(regID, changeScenarioName)
#produceSummaryFromRegContrib(regID, predevScenarioName)

##Fitzroy
#regID = 'FI'
#produceSummaryFromRegContrib(regID, baselineScenarioName)
#produceSummaryFromRegContrib(regID, changeScenarioName)
#produceSummaryFromRegContrib(regID, predevScenarioName)

##Mackay
#regID = 'MW'
#produceSummaryFromRegContrib(regID, baselineScenarioName)
#produceSummaryFromRegContrib(regID, changeScenarioName)
#produceSummaryFromRegContrib(regID, predevScenarioName)

##Wet Tropics
#regID = 'WT'
#produceSummaryFromRegContrib(regID, baselineScenarioName)
#produceSummaryFromRegContrib(regID, changeScenarioName)
#produceSummaryFromRegContrib(regID, predevScenarioName)

print("Finished first summary")

Processing \\athenasmb\pdreef\RC9_RC2017\AdditionalScenarios\Results\ReportCardSpan_1986_2014\BM_RC9_ScenarioBaseline_1986_2014.7z Scenario: ScenarioBaseline
BM_RC9_ScenarioBaseline_1986_2014/RegContributorDataGrid.csv
Saved P:\projects\RC9_ResultsSets\Tester\out\BM_RC2017_ScenarioBaseline_RegToOutlet.csv
Processing \\athenasmb\pdreef\RC9_RC2017\AdditionalScenarios\Results\ReportCardSpan_1986_2014\BM_RC9_Scenario_DtoC_1986_2014.7z Scenario: Scenario_DtoC
BM_RC9_Scenario_DtoC_1986_2014/RegContributorDataGrid.csv
Saved P:\projects\RC9_ResultsSets\Tester\out\BM_RC2017_Scenario_DtoC_RegToOutlet.csv
Processing \\athenasmb\pdreef\RC9_RC2017\AdditionalScenarios\Results\ReportCardSpan_1986_2014\BM_RC9_Scenario_AllB_1986_2014.7z Scenario: Scenario_AllB
BM_RC9_Scenario_AllB_1986_2014/RegContributorDataGrid.csv
Saved P:\projects\RC9_ResultsSets\Tester\out\BM_RC2017_Scenario_AllB_RegToOutlet.csv
Processing \\athenasmb\pdreef\RC9_RC2017\AdditionalScenarios\Results\ReportCardSpan_1986_2014\BM_RC9_Sc

In [5]:
#Build relationships of constituent names to groups
#tssGrp = 'TSS'
#tnGrp = 'TN'
#tpGrp = 'TP'
#ps2Grp = 'PSII'
#nonps2Grp = 'non-PSII'
##Nope, will read in a CSV to DataFrame instead, thus joing will 'drop' flow from this comparison
#constToGrp = {'Sediment - Fine':tssGrp}

constToGrp = pd.read_csv(pathToContstitsToGrpFile)

baseSupplyStr = 'Base_Supply_KG'
baseExportStr = 'Base_Export_KG'
D2CSupplyStr = 'D2C_Supply_KG'
D2CExportStr = 'D2C_Export_KG'
D2CPercRedStr = 'D2C_PercRed'
AllBSupplyStr = 'AllB_Supply_KG'
AllBExportStr = 'AllB_Export_KG'
AllBPercRedStr = 'AllB_PercRed'
AllASupplyStr = 'AllA_Supply_KG'
AllAExportStr = 'AllA_Export_KG'
AllAPercRedStr = 'AllA_PercRed'
predevSupplyStr = 'PreDev_Supply_KG'
predevExportStr = 'PreDev_Export_KG'
repRegStr = 'Rep_Region'
sourceRegStr = 'Source Region'
constituentStr = 'Constituent'
fuStr = 'FU'
groupStr = 'GROUP'
modelRegStr = 'MODELREG'
#percRedStr = 'PercReduction'
regTotalStr = 'REGIONAL TOTAL'

#allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])
#allExportsAndReductions = pd.DataFrame()


In [6]:
def getPathInfoForReductions(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '_RC9_' + scenarioString + '_1986_2014.7z'
    fileOut = outPath + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut

In [7]:
def getPreDevPathInfo(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '_RC9_' + scenarioString + '_1986_2014.7z'
    fileOut = preDevCSVLoc + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut

In [8]:
#So now we would be reading in known summary files

def calculateReductions(regName, thisRegID):
    
    unneededFile, theBaseSummaryFile = getPathInfoForReductions(thisRegID, baseFileName)
    unneededFile, theD2CSummaryFile = getPathInfoForReductions(thisRegID, D2CFileName)
    unneededFile, theAllBSummaryFile = getPathInfoForReductions(thisRegID, AllBFileName)
    unneededFile, theAllASummaryFile = getPathInfoForReductions(thisRegID, AllAFileName)
    unneededFile, thePreDevSummaryFile = getPreDevPathInfo(thisRegID, predevScenarioName)
    
    #print("Reading: " + theBaseSummaryFile)
    baseSummaryDF = pd.read_csv(theBaseSummaryFile)
    #print("Reading: " + theChangeSummaryFile)
    D2CSummaryDF = pd.read_csv(theD2CSummaryFile)
    AllBSummaryDF = pd.read_csv(theAllBSummaryFile)
    AllASummaryDF = pd.read_csv(theAllASummaryFile)
    #print("Reading: " + thePreDevSummaryFile)
    predevSummaryDF = pd.read_csv(thePreDevSummaryFile)
    
    #inner drops out those that don't merge, left keep them
    baseTotals = pd.merge(baseSummaryDF.groupby([repRegStr,constituentStr,fuStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum', 'AreaHA':'first'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    baseTotals = baseTotals.rename(columns={repRegStr:sourceRegStr,'AreaHA':'AREA_HA',annLoadToStream:baseSupplyStr, annLoadToExport:baseExportStr})
    baseTotals = baseTotals[[sourceRegStr, groupStr, constituentStr, fuStr, 'AREA_HA', baseSupplyStr, baseExportStr]]
    #baseTotals.head(40)
    
    predevTotals = pd.merge(predevSummaryDF.groupby([repRegStr,constituentStr,fuStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    predevTotals = predevTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:predevSupplyStr, annLoadToExport:predevExportStr})
    predevTotals = predevTotals[[sourceRegStr, constituentStr, fuStr, predevSupplyStr, predevExportStr]]
    
    D2CTotals = pd.merge(D2CSummaryDF.groupby([repRegStr,constituentStr,fuStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    D2CTotals = D2CTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:D2CSupplyStr, annLoadToExport:D2CExportStr})
    D2CTotals = D2CTotals[[sourceRegStr, constituentStr, fuStr, D2CSupplyStr, D2CExportStr]]
    
    AllBTotals = pd.merge(AllBSummaryDF.groupby([repRegStr,constituentStr,fuStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    AllBTotals = AllBTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:AllBSupplyStr, annLoadToExport:AllBExportStr})
    AllBTotals = AllBTotals[[sourceRegStr, constituentStr, fuStr, AllBSupplyStr, AllBExportStr]]
    
    AllATotals = pd.merge(AllASummaryDF.groupby([repRegStr,constituentStr,fuStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    AllATotals = AllATotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:AllASupplyStr, annLoadToExport:AllAExportStr})
    AllATotals = AllATotals[[sourceRegStr, constituentStr, fuStr, AllASupplyStr, AllAExportStr]]
    
    #changeTotals.head(8)
    
    reductTotalsDF = [D2CTotals, AllBTotals, AllATotals]
    reductionsMerged = reduce(lambda  left,right: pd.merge(left,right,on=[sourceRegStr, constituentStr, fuStr], how='outer'), reductTotalsDF)
    
    #, columns=['Rep_Region', 'GROUP', 'Constituent', 'FU', 'AREA_HA', 'Base_Supply_KG', 'Base_Export_KG', 'PreDev_Supply_KG', 'PreDev_Export_KG', 'Change_Supply_KG', 'Change_Export_KG']
    anthroRegByLuse = pd.merge(pd.merge(baseTotals, predevTotals, how='left', left_on=[sourceRegStr,constituentStr,fuStr], right_on = [sourceRegStr,constituentStr,fuStr]).reset_index(), reductionsMerged, how='left', left_on=[sourceRegStr,constituentStr,fuStr], right_on = [sourceRegStr,constituentStr,fuStr])
    #anthroRegByLuse.head(8)
    
    theFileOut = outPath + '\\' + summariesOutFolder + '\\' + thisRegID + '_RC2017_AddScenarios_AnthroRegByLanduse.csv'
    anthroRegByLuse.to_csv(theFileOut, index=False)
    #print("Saved " + theFileOut)
    
    regExports = anthroRegByLuse.groupby([sourceRegStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', D2CExportStr:'sum', AllBExportStr:'sum', AllAExportStr:'sum'})
    
    regExports[modelRegStr] = regName
    regExports[D2CPercRedStr] = (regExports[baseExportStr] - regExports[D2CExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports[AllBPercRedStr] = (regExports[baseExportStr] - regExports[AllBExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports[AllAPercRedStr] = (regExports[baseExportStr] - regExports[AllAExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports = regExports.reset_index()
    regExports = regExports[[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr,
                             D2CExportStr, D2CPercRedStr, AllBExportStr, AllBPercRedStr, AllAPercRedStr, AllAExportStr]]
    #regExports.head(15)
    
    modelRegExports = regExports.groupby([modelRegStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', D2CExportStr:'sum', AllBExportStr:'sum', AllAExportStr:'sum'})
    modelRegExports[sourceRegStr] = regTotalStr
    modelRegExports[D2CPercRedStr] = (modelRegExports[baseExportStr] - modelRegExports[D2CExportStr]).div(modelRegExports[baseExportStr] - modelRegExports[predevExportStr]).mul(100)
    modelRegExports[AllBPercRedStr] = (modelRegExports[baseExportStr] - modelRegExports[AllBExportStr]).div(modelRegExports[baseExportStr] - modelRegExports[predevExportStr]).mul(100)
    modelRegExports[AllAPercRedStr] = (modelRegExports[baseExportStr] - modelRegExports[AllAExportStr]).div(modelRegExports[baseExportStr] - modelRegExports[predevExportStr]).mul(100)
    modelRegExports = modelRegExports.reset_index()
    modelRegExports = modelRegExports[[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr,
                                       D2CExportStr, D2CPercRedStr, AllBExportStr, AllBPercRedStr, AllAPercRedStr, AllAExportStr]]
    ###modelRegExports.head(15)
    regExports = regExports.append(modelRegExports, ignore_index=True)
    
    return regExports


In [9]:
allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr,
                                                baseExportStr, D2CExportStr, D2CPercRedStr, AllBExportStr, AllBPercRedStr,
                                                AllAExportStr, AllAPercRedStr])

#theseExports = calculateReductions('Burdekin', 'BU')
#allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Burnett Mary', 'BM')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Cape York', 'CY')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Fitzroy', 'FI')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Mackay Whitsunday', 'MW')#allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Wet Tropics', 'WT')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theFileNameOut = outPath + '\\' + summariesOutFolder + '\\AllExportsAndReductions.csv'
allExportsAndReductions.to_csv(theFileNameOut, index=False)
allExportsAndReductions.head(20)
#print(allExportsAndReductions)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,AllA_Export_KG,AllA_PercRed,AllB_Export_KG,AllB_PercRed,Base_Export_KG,Constituent,D2C_Export_KG,D2C_PercRed,GROUP,MODELREG,PreDev_Export_KG,Source Region
0,6.732250,0.000000,6.732250,0.000000,6.732250,24-D,6.732250,0.000000,non-PSII,Burnett Mary,0.000000,Baffle
1,8.543583,0.000000,8.543583,0.000000,8.543583,Atrazine,8.543583,0.000000,PSII,Burnett Mary,0.000000,Baffle
2,5.599603,0.000000,5.599603,0.000000,5.599603,Diuron,5.599603,0.000000,PSII,Burnett Mary,0.000000,Baffle
3,4.985507,0.000000,4.985507,0.000000,4.985507,Fluroxypyr,4.985507,0.000000,non-PSII,Burnett Mary,0.000000,Baffle
4,2.328639,0.000000,2.328639,0.000000,2.328639,Glyphosate,2.328639,0.000000,non-PSII,Burnett Mary,0.000000,Baffle
5,0.000000,NaN,0.000000,NaN,0.000000,Haloxyfop,0.000000,NaN,non-PSII,Burnett Mary,0.000000,Baffle
6,0.000000,NaN,0.000000,NaN,0.000000,Hexazinone,0.000000,NaN,PSII,Burnett Mary,0.000000,Baffle
7,0.102374,0.000000,0.102374,0.000000,0.102374,Imazapic,0.102374,0.000000,non-PSII,Burnett Mary,0.000000,Baffle
8,0.017768,0.000000,0.017768,0.000000,0.017768,MCPA,0.017768,0.000000,non-PSII,Burnett Mary,0.000000,Baffle
9,1.228279,0.000000,1.228279,0.000000,1.228279,Metribuzin,1.228279,0.000000,non-priority-PSII,Burnett Mary,0.000000,Baffle
